In [69]:
import pandas as pd
import json
from itertools import groupby

In [101]:
# load data
with open('../data/games.json', 'r') as f:
    games = json.load(f)
    
actions = []

yes = 0
no = 0
for game in games:
    for quarter in game['quarters']:
        for play in quarter['plays']:
            for action in play['actions']:
                if 'athlete' in action:
                    action['athlete'] = action['athlete']['$oid']
                    yes += 1
                    actions.append(action)
                else:
                    no += 1
                
print(yes)
print(no)

1776
316


In [136]:
#one must sort before grouping
actions_sorted = sorted(actions, key = lambda a: a['athlete'])
actions_grouped = groupby(actions_sorted, lambda a: a['athlete'])
[{'athlete': key, 'actions': list(group)} for key, group in actions_grouped]

[{'actions': [{'action_type': 'receiver',
    'athlete': '56a6d947634b6675bab2308e',
    'observations': {'1D': '1',
     'AirYds': '5',
     'Cont': '',
     'Drop': '',
     'EZ': '',
     'Fum': '',
     'MT': '1',
     'Note': '',
     'Rec': '1',
     'TD': '',
     'Tgt': '1',
     'Tyds': '10',
     'YAC': '5',
     'type': 'receiver'}},
   {'action_type': 'receiver',
    'athlete': '56a6d947634b6675bab2308e',
    'observations': {'1D': '',
     'AirYds': '19',
     'Cont': '1',
     'Drop': '',
     'EZ': '',
     'Fum': '',
     'MT': '',
     'Note': '',
     'Rec': '',
     'TD': '',
     'Tgt': '1',
     'Tyds': '',
     'YAC': '',
     'type': 'receiver'}},
   {'action_type': 'receiver',
    'athlete': '56a6d947634b6675bab2308e',
    'observations': {'1D': '',
     'AirYds': '',
     'Cont': '1',
     'Drop': '',
     'EZ': '',
     'Fum': '',
     'MT': '',
     'Note': '',
     'Rec': '',
     'TD': '',
     'Tgt': '1',
     'Tyds': '',
     'YAC': '',
     'type': 'rece